In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[15,6]

pd.set_option('display.max_columns',50)  # display the data

In [2]:
train=pd.read_csv(r"C:\Users\Lenovo\Downloads\Train janta hack\Train\Train.csv")
test=pd.read_csv(r"C:\Users\Lenovo\Downloads\test janta hack.csv")

fhc=pd.read_csv(r"C:\Users\Lenovo\Downloads\Train janta hack\Train\First_Health_Camp_Attended.csv")

shc=pd.read_csv(r"C:\Users\Lenovo\Downloads\Train janta hack\Train\Second_Health_Camp_Attended.csv")

thc=pd.read_csv(r"C:\Users\Lenovo\Downloads\Train janta hack\Train\Third_Health_Camp_Attended.csv")

hcdetail=pd.read_csv(r"C:\Users\Lenovo\Downloads\Train janta hack\Train\Health_Camp_Detail.csv")

pp=pd.read_csv(r"C:\Users\Lenovo\Downloads\Train janta hack\Train\Patient_Profile.csv")

In [3]:
#checking shape
train.shape,test.shape

((75278, 8), (35249, 8))

In [4]:
#combine train and test
combined=pd.concat([train,test],ignore_index=True)

In [5]:
combined.shape

(110527, 8)

In [6]:
combined.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [7]:
# fhc with combined data
combined=pd.merge(left=combined,right=fhc,on=['Patient_ID','Health_Camp_ID'],how='left')

In [8]:
#shc and combine
combined=pd.merge(left=combined,right=shc,on=['Patient_ID','Health_Camp_ID'],how='left')

In [9]:
#thc an combined
combined=pd.merge(left=combined,right=thc,on=['Patient_ID','Health_Camp_ID'],how='left')

In [10]:
combined.shape

(110527, 14)

In [11]:
hcdetail.head()

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2
2,6544,03-Nov-03,15-Nov-03,First,F,1
3,6585,22-Nov-03,05-Dec-03,First,E,2
4,6561,30-Nov-03,18-Dec-03,First,E,1


In [12]:
#hcdetails and combined
combined=pd.merge(left=combined,right=hcdetail,on='Health_Camp_ID',how='left')

In [13]:
pp.head(2)

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry


In [14]:
#pp and combined
combined=pd.merge(left=combined,right=pp,on='Patient_ID',how='left')

In [15]:
combined.shape


(110527, 29)

In [16]:
combined.isnull().sum()

Patient_ID                        0
Health_Camp_ID                    0
Registration_Date               334
Var1                              0
Var2                              0
Var3                              0
Var4                              0
Var5                              0
Donation                     104309
Health_Score                 104309
Unnamed: 4                   110527
Health Score                 102708
Number_of_stall_visited      104012
Last_Stall_Visited_Number    104012
Camp_Start_Date                   0
Camp_End_Date                     0
Category1                         0
Category2                         0
Category3                         0
Online_Follower                   0
LinkedIn_Shared                   0
Twitter_Shared                    0
Facebook_Shared                   0
Income                            0
Education_Score                   0
Age                               0
First_Interaction                 0
City_Type                   

In [17]:
combined['Registration_Date']=combined['Registration_Date'].fillna(combined['Registration_Date'].mode()[0])

In [18]:
combined.isnull().sum()

Patient_ID                        0
Health_Camp_ID                    0
Registration_Date                 0
Var1                              0
Var2                              0
Var3                              0
Var4                              0
Var5                              0
Donation                     104309
Health_Score                 104309
Unnamed: 4                   110527
Health Score                 102708
Number_of_stall_visited      104012
Last_Stall_Visited_Number    104012
Camp_Start_Date                   0
Camp_End_Date                     0
Category1                         0
Category2                         0
Category3                         0
Online_Follower                   0
LinkedIn_Shared                   0
Twitter_Shared                    0
Facebook_Shared                   0
Income                            0
Education_Score                   0
Age                               0
First_Interaction                 0
City_Type                   

## Feature Engineering

In [19]:
# registration date
#code to conver string into date
pd.to_datetime(combined.Registration_Date,dayfirst=True)
combined['Registration_Date']=pd.to_datetime(combined.Registration_Date,dayfirst=True)
combined['Registration_Day']=combined.Registration_Date.dt.day
combined['Registration_Month']=combined.Registration_Date.dt.month
combined['Registration_Year']=combined.Registration_Date.dt.year

In [20]:
# camp start
combined['Camp_Start_Month']=pd.DatetimeIndex(combined.Camp_Start_Date).month
combined['Camp_Start_Year']=pd.DatetimeIndex(combined.Camp_Start_Date).year
combined['Camp_Start_Day']=pd.DatetimeIndex(combined.Camp_Start_Date).day

#camp end
combined['Camp_End_Month']=pd.DatetimeIndex(combined.Camp_End_Date).month
combined['Camp_End_Year']=pd.DatetimeIndex(combined.Camp_End_Date).year
combined['Camp_End_Day']=pd.DatetimeIndex(combined.Camp_End_Date).day

In [21]:
#lets create new feature from patient profile based on online presence
combined['Online_Presence']=combined['Online_Follower']+combined['Twitter_Shared']+combined['Facebook_Shared']+combined['LinkedIn_Shared']

In [22]:
#dropping these four now
combined=combined.drop(['LinkedIn_Shared','Twitter_Shared','Online_Follower','Facebook_Shared'],axis=1)

In [23]:
#camp duration from camp end date and camp start date

combined['Camp_Start_Date']=pd.to_datetime(combined.Camp_Start_Date,dayfirst=True)

combined['Camp_End_Date']=pd.to_datetime(combined.Camp_End_Date,dayfirst=True)
combined['Camp_Duration']=(combined['Camp_End_Date']-combined['Camp_Start_Date']).dt.days

In [24]:
# days interation diff =registration date-first interation

# converting First interaction date into date time format

combined['First_Interaction']=pd.to_datetime(combined.First_Interaction,dayfirst=True)

combined['Int_Days_Diff']=(combined['Registration_Date']-combined['First_Interaction']).dt.days

In [25]:
combined.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Donation,Health_Score,Unnamed: 4,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Registration_Day,Registration_Month,Registration_Year,Camp_Start_Month,Camp_Start_Year,Camp_Start_Day,Camp_End_Month,Camp_End_Year,Camp_End_Day,Online_Presence,Camp_Duration,Int_Days_Diff
0,489652,6578,2005-09-10,4,0,0,0,2,NaN,NaN,NaN,NaN,2.0,1.0,2005-08-16,2005-10-14,Third,G,2,None,None,None,2004-12-06,NaN,NaN,10,9,2005,8,2005,16,10,2005,14,0,59,278
1,507246,6578,2005-08-18,45,5,0,0,7,NaN,NaN,NaN,NaN,NaN,NaN,2005-08-16,2005-10-14,Third,G,2,1,75,40,2004-09-08,C,Others,18,8,2005,8,2005,16,10,2005,14,0,59,344
2,523729,6534,2006-04-29,0,0,0,0,0,NaN,NaN,NaN,0.402054,NaN,NaN,2005-10-17,2007-11-07,Second,A,2,None,None,None,2004-06-22,NaN,NaN,29,4,2006,10,2005,17,11,2007,7,0,751,676
3,524931,6535,2004-02-07,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2004-02-01,2004-02-18,First,E,2,None,None,None,2004-02-07,I,NaN,7,2,2004,2,2004,1,2,2004,18,0,17,0
4,521364,6529,2006-02-28,15,1,0,0,7,NaN,NaN,NaN,0.845597,NaN,NaN,2006-03-30,2006-04-03,Second,A,2,1,70,40,2003-07-04,I,Technology,28,2,2006,3,2006,30,4,2006,3,1,4,970


In [26]:
# unique patient in health camp 
# nunique will give us the count of unique patient in each health camp
combined['HC_per_patient']=combined.groupby('Patient_ID')['Health_Camp_ID'].transform('nunique')

combined['Patient_per_HC']=combined.groupby('Health_Camp_ID')['Patient_ID'].transform('nunique')

In [27]:
combined.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Donation,Health_Score,Unnamed: 4,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Registration_Day,Registration_Month,Registration_Year,Camp_Start_Month,Camp_Start_Year,Camp_Start_Day,Camp_End_Month,Camp_End_Year,Camp_End_Day,Online_Presence,Camp_Duration,Int_Days_Diff,HC_per_patient,Patient_per_HC
0,489652,6578,2005-09-10,4,0,0,0,2,NaN,NaN,NaN,NaN,2.0,1.0,2005-08-16,2005-10-14,Third,G,2,None,None,None,2004-12-06,NaN,NaN,10,9,2005,8,2005,16,10,2005,14,0,59,278,11,2837
1,507246,6578,2005-08-18,45,5,0,0,7,NaN,NaN,NaN,NaN,NaN,NaN,2005-08-16,2005-10-14,Third,G,2,1,75,40,2004-09-08,C,Others,18,8,2005,8,2005,16,10,2005,14,0,59,344,26,2837
2,523729,6534,2006-04-29,0,0,0,0,0,NaN,NaN,NaN,0.402054,NaN,NaN,2005-10-17,2007-11-07,Second,A,2,None,None,None,2004-06-22,NaN,NaN,29,4,2006,10,2005,17,11,2007,7,0,751,676,7,3597
3,524931,6535,2004-02-07,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2004-02-01,2004-02-18,First,E,2,None,None,None,2004-02-07,I,NaN,7,2,2004,2,2004,1,2,2004,18,0,17,0,6,1882
4,521364,6529,2006-02-28,15,1,0,0,7,NaN,NaN,NaN,0.845597,NaN,NaN,2006-03-30,2006-04-03,Second,A,2,1,70,40,2003-07-04,I,Technology,28,2,2006,3,2006,30,4,2006,3,1,4,970,23,3823


In [28]:
#unique patients visited in the beginning of the year camp started

combined['Patient_Per_Year']=combined.groupby('Camp_Start_Year')['Patient_ID'].transform('nunique')

combined.groupby('Camp_Start_Year')['Patient_ID'].nunique()

Camp_Start_Year
2003     3081
2004    10902
2005    22359
2006    16175
2007     2579
Name: Patient_ID, dtype: int64

In [29]:
# patient per month

combined['Patient_Per_Month']=combined.groupby('Camp_Start_Month')['Patient_ID'].transform('nunique')

combined.groupby('Camp_Start_Month')['Patient_ID'].nunique()

Camp_Start_Month
1      9076
2      8717
3      3823
4      6164
5      1903
6      8208
7      3726
8     10445
9     11210
10     8242
11     8757
12     6774
Name: Patient_ID, dtype: int64

In [30]:
# patient in end year
combined['Patients_Per_End_Year']=combined.groupby('Camp_End_Year')['Patient_ID'].transform('nunique')

combined['Patients_Per_End_Month']=combined.groupby('Camp_End_Month')['Patient_ID'].transform('nunique')


In [31]:
combined.shape

(110527, 43)

## Target variable

In [32]:
def outcome(a,b,c,d):
    if((a>0)|(b>0)|(c>0)|(d>0)):
        return(1)
    else:
        return(0)

In [33]:
#generating target
combined['Target']=combined.apply(lambda x:outcome(x['Health_Score'],x['Health Score'],x['Number_of_stall_visited'],x['Last_Stall_Visited_Number']),axis=1)

In [34]:
combined.Target.value_counts()

0    89993
1    20534
Name: Target, dtype: int64

In [35]:
#drop unnecessary columns
combined.columns

Index(['Patient_ID', 'Health_Camp_ID', 'Registration_Date', 'Var1', 'Var2',
       'Var3', 'Var4', 'Var5', 'Donation', 'Health_Score', 'Unnamed: 4',
       'Health Score', 'Number_of_stall_visited', 'Last_Stall_Visited_Number',
       'Camp_Start_Date', 'Camp_End_Date', 'Category1', 'Category2',
       'Category3', 'Income', 'Education_Score', 'Age', 'First_Interaction',
       'City_Type', 'Employer_Category', 'Registration_Day',
       'Registration_Month', 'Registration_Year', 'Camp_Start_Month',
       'Camp_Start_Year', 'Camp_Start_Day', 'Camp_End_Month', 'Camp_End_Year',
       'Camp_End_Day', 'Online_Presence', 'Camp_Duration', 'Int_Days_Diff',
       'HC_per_patient', 'Patient_per_HC', 'Patient_Per_Year',
       'Patient_Per_Month', 'Patients_Per_End_Year', 'Patients_Per_End_Month',
       'Target'],
      dtype='object')

In [36]:
new=combined.drop(['Patient_ID', 'Health_Camp_ID', 'Registration_Date','Donation', 'Health_Score', 'Unnamed: 4',
                  'Health Score', 'Number_of_stall_visited', 'Last_Stall_Visited_Number',
                   'Camp_Start_Date', 'Camp_End_Date','Income', 'Education_Score','First_Interaction',
       'City_Type', 'Employer_Category','Age'],axis=1)

In [37]:
# to save memory we will delete the rest
del combined,fhc,shc,thc,hcdetail

In [38]:
new.columns

Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Category1', 'Category2',
       'Category3', 'Registration_Day', 'Registration_Month',
       'Registration_Year', 'Camp_Start_Month', 'Camp_Start_Year',
       'Camp_Start_Day', 'Camp_End_Month', 'Camp_End_Year', 'Camp_End_Day',
       'Online_Presence', 'Camp_Duration', 'Int_Days_Diff', 'HC_per_patient',
       'Patient_per_HC', 'Patient_Per_Year', 'Patient_Per_Month',
       'Patients_Per_End_Year', 'Patients_Per_End_Month', 'Target'],
      dtype='object')

In [39]:
new.shape

(110527, 27)

In [40]:
# category 1
mapped={'First':1,'Second':2,'Third':3}

new['Category1']=new.Category1.map(mapped)

In [41]:
#category 2
new['Category2']=pd.factorize(new.Category2,sort=True)[0]

# factorize works same as label encoder

In [42]:
new.shape

(110527, 27)

In [43]:
new.isnull().sum()

Var1                      0
Var2                      0
Var3                      0
Var4                      0
Var5                      0
Category1                 0
Category2                 0
Category3                 0
Registration_Day          0
Registration_Month        0
Registration_Year         0
Camp_Start_Month          0
Camp_Start_Year           0
Camp_Start_Day            0
Camp_End_Month            0
Camp_End_Year             0
Camp_End_Day              0
Online_Presence           0
Camp_Duration             0
Int_Days_Diff             0
HC_per_patient            0
Patient_per_HC            0
Patient_Per_Year          0
Patient_Per_Month         0
Patients_Per_End_Year     0
Patients_Per_End_Month    0
Target                    0
dtype: int64

In [44]:
train.shape,test.shape

((75278, 8), (35249, 8))

## Split data back in Train and test and delete target

In [48]:
newtrain=new.loc[0:train.shape[0]-1,:]
newtest=new.loc[train.shape[0]:,:]

In [49]:
# drop target from test
newtest=newtest.drop('Target',axis=1)

In [50]:
newtrain.shape,newtest.shape

((75278, 27), (35249, 26))

# Model Building
## Logistic Regression
## Random Forest
## Gradient Boosting
## Xtreme Gradient Boosting
## Catboost 


In [54]:
X=newtrain.drop('Target',axis=1)
y=newtrain.Target

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [56]:
#models
lr=LogisticRegression()
rf=RandomForestClassifier()
gbm=GradientBoostingClassifier()
xgb=XGBClassifier()
cboost=CatBoostClassifier()

In [59]:
# building voting Classifier model
from sklearn.ensemble import VotingClassifier
vc=VotingClassifier(estimators=[('lr',lr),('rf',rf),('gbm',gbm),('xgb',xgb),('cb',cboost)],voting='soft')

In [60]:
pred=vc.fit(X,y).predict_proba(newtest)

Learning rate set to 0.065204
0:	learn: 0.6554568	total: 29.3ms	remaining: 29.3s
1:	learn: 0.6239134	total: 58.8ms	remaining: 29.3s
2:	learn: 0.5949326	total: 88.9ms	remaining: 29.6s
3:	learn: 0.5724263	total: 117ms	remaining: 29s
4:	learn: 0.5536839	total: 147ms	remaining: 29.2s
5:	learn: 0.5365705	total: 174ms	remaining: 28.8s
6:	learn: 0.5223670	total: 214ms	remaining: 30.4s
7:	learn: 0.5115359	total: 245ms	remaining: 30.3s
8:	learn: 0.5003178	total: 276ms	remaining: 30.3s
9:	learn: 0.4913630	total: 307ms	remaining: 30.3s
10:	learn: 0.4825368	total: 337ms	remaining: 30.3s
11:	learn: 0.4762111	total: 367ms	remaining: 30.2s
12:	learn: 0.4701226	total: 398ms	remaining: 30.2s
13:	learn: 0.4654361	total: 435ms	remaining: 30.6s
14:	learn: 0.4604015	total: 467ms	remaining: 30.7s
15:	learn: 0.4561663	total: 494ms	remaining: 30.4s
16:	learn: 0.4518558	total: 526ms	remaining: 30.4s
17:	learn: 0.4490944	total: 554ms	remaining: 30.2s
18:	learn: 0.4463778	total: 583ms	remaining: 30.1s
19:	learn:

165:	learn: 0.4002658	total: 4.81s	remaining: 24.2s
166:	learn: 0.4001314	total: 4.84s	remaining: 24.1s
167:	learn: 0.4000535	total: 4.86s	remaining: 24.1s
168:	learn: 0.3999189	total: 4.89s	remaining: 24.1s
169:	learn: 0.3997582	total: 4.92s	remaining: 24s
170:	learn: 0.3996917	total: 4.95s	remaining: 24s
171:	learn: 0.3995920	total: 4.97s	remaining: 23.9s
172:	learn: 0.3995132	total: 5s	remaining: 23.9s
173:	learn: 0.3993921	total: 5.03s	remaining: 23.9s
174:	learn: 0.3993248	total: 5.06s	remaining: 23.8s
175:	learn: 0.3992166	total: 5.08s	remaining: 23.8s
176:	learn: 0.3990202	total: 5.11s	remaining: 23.8s
177:	learn: 0.3989331	total: 5.14s	remaining: 23.7s
178:	learn: 0.3988624	total: 5.17s	remaining: 23.7s
179:	learn: 0.3987406	total: 5.19s	remaining: 23.7s
180:	learn: 0.3986587	total: 5.22s	remaining: 23.6s
181:	learn: 0.3985696	total: 5.25s	remaining: 23.6s
182:	learn: 0.3984973	total: 5.28s	remaining: 23.6s
183:	learn: 0.3983855	total: 5.31s	remaining: 23.5s
184:	learn: 0.39833

328:	learn: 0.3866049	total: 9.58s	remaining: 19.5s
329:	learn: 0.3864593	total: 9.61s	remaining: 19.5s
330:	learn: 0.3863965	total: 9.63s	remaining: 19.5s
331:	learn: 0.3863408	total: 9.66s	remaining: 19.4s
332:	learn: 0.3863076	total: 9.68s	remaining: 19.4s
333:	learn: 0.3862448	total: 9.71s	remaining: 19.4s
334:	learn: 0.3862193	total: 9.73s	remaining: 19.3s
335:	learn: 0.3861833	total: 9.76s	remaining: 19.3s
336:	learn: 0.3861252	total: 9.79s	remaining: 19.3s
337:	learn: 0.3860530	total: 9.82s	remaining: 19.2s
338:	learn: 0.3860191	total: 9.84s	remaining: 19.2s
339:	learn: 0.3859587	total: 9.87s	remaining: 19.2s
340:	learn: 0.3858893	total: 9.9s	remaining: 19.1s
341:	learn: 0.3858470	total: 9.93s	remaining: 19.1s
342:	learn: 0.3857907	total: 9.95s	remaining: 19.1s
343:	learn: 0.3857285	total: 9.98s	remaining: 19s
344:	learn: 0.3856707	total: 10s	remaining: 19s
345:	learn: 0.3856190	total: 10s	remaining: 19s
346:	learn: 0.3855390	total: 10.1s	remaining: 18.9s
347:	learn: 0.3854211	t

491:	learn: 0.3772132	total: 14.7s	remaining: 15.2s
492:	learn: 0.3771958	total: 14.7s	remaining: 15.2s
493:	learn: 0.3771383	total: 14.8s	remaining: 15.1s
494:	learn: 0.3770709	total: 14.8s	remaining: 15.1s
495:	learn: 0.3770313	total: 14.8s	remaining: 15.1s
496:	learn: 0.3769201	total: 14.9s	remaining: 15s
497:	learn: 0.3769007	total: 14.9s	remaining: 15s
498:	learn: 0.3768700	total: 14.9s	remaining: 15s
499:	learn: 0.3768269	total: 14.9s	remaining: 14.9s
500:	learn: 0.3767844	total: 15s	remaining: 14.9s
501:	learn: 0.3767604	total: 15s	remaining: 14.9s
502:	learn: 0.3767161	total: 15s	remaining: 14.8s
503:	learn: 0.3766676	total: 15.1s	remaining: 14.8s
504:	learn: 0.3766106	total: 15.1s	remaining: 14.8s
505:	learn: 0.3765464	total: 15.1s	remaining: 14.7s
506:	learn: 0.3765040	total: 15.1s	remaining: 14.7s
507:	learn: 0.3764681	total: 15.2s	remaining: 14.7s
508:	learn: 0.3764222	total: 15.2s	remaining: 14.7s
509:	learn: 0.3763551	total: 15.2s	remaining: 14.6s
510:	learn: 0.3762868	to

657:	learn: 0.3694866	total: 19.9s	remaining: 10.4s
658:	learn: 0.3694319	total: 20s	remaining: 10.3s
659:	learn: 0.3693929	total: 20s	remaining: 10.3s
660:	learn: 0.3693704	total: 20s	remaining: 10.3s
661:	learn: 0.3693247	total: 20s	remaining: 10.2s
662:	learn: 0.3692992	total: 20.1s	remaining: 10.2s
663:	learn: 0.3692697	total: 20.1s	remaining: 10.2s
664:	learn: 0.3692283	total: 20.1s	remaining: 10.1s
665:	learn: 0.3691695	total: 20.1s	remaining: 10.1s
666:	learn: 0.3691402	total: 20.2s	remaining: 10.1s
667:	learn: 0.3691111	total: 20.2s	remaining: 10s
668:	learn: 0.3690579	total: 20.2s	remaining: 10s
669:	learn: 0.3690246	total: 20.2s	remaining: 9.97s
670:	learn: 0.3689960	total: 20.3s	remaining: 9.94s
671:	learn: 0.3689650	total: 20.3s	remaining: 9.91s
672:	learn: 0.3689209	total: 20.3s	remaining: 9.88s
673:	learn: 0.3688858	total: 20.4s	remaining: 9.85s
674:	learn: 0.3688467	total: 20.4s	remaining: 9.82s
675:	learn: 0.3687938	total: 20.4s	remaining: 9.78s
676:	learn: 0.3687562	to

823:	learn: 0.3626259	total: 24.6s	remaining: 5.26s
824:	learn: 0.3625921	total: 24.6s	remaining: 5.23s
825:	learn: 0.3625685	total: 24.7s	remaining: 5.2s
826:	learn: 0.3625328	total: 24.7s	remaining: 5.17s
827:	learn: 0.3625116	total: 24.7s	remaining: 5.13s
828:	learn: 0.3624803	total: 24.8s	remaining: 5.11s
829:	learn: 0.3624231	total: 24.8s	remaining: 5.08s
830:	learn: 0.3623801	total: 24.8s	remaining: 5.04s
831:	learn: 0.3623209	total: 24.8s	remaining: 5.02s
832:	learn: 0.3622854	total: 24.9s	remaining: 4.99s
833:	learn: 0.3622496	total: 24.9s	remaining: 4.96s
834:	learn: 0.3621796	total: 24.9s	remaining: 4.92s
835:	learn: 0.3621352	total: 25s	remaining: 4.89s
836:	learn: 0.3620995	total: 25s	remaining: 4.87s
837:	learn: 0.3620458	total: 25s	remaining: 4.83s
838:	learn: 0.3620008	total: 25s	remaining: 4.8s
839:	learn: 0.3619692	total: 25.1s	remaining: 4.78s
840:	learn: 0.3619446	total: 25.1s	remaining: 4.75s
841:	learn: 0.3619063	total: 25.1s	remaining: 4.72s
842:	learn: 0.3618717	

985:	learn: 0.3566998	total: 29.8s	remaining: 424ms
986:	learn: 0.3566589	total: 29.9s	remaining: 393ms
987:	learn: 0.3566355	total: 29.9s	remaining: 363ms
988:	learn: 0.3566180	total: 29.9s	remaining: 333ms
989:	learn: 0.3565808	total: 30s	remaining: 303ms
990:	learn: 0.3565558	total: 30s	remaining: 272ms
991:	learn: 0.3565215	total: 30s	remaining: 242ms
992:	learn: 0.3564935	total: 30s	remaining: 212ms
993:	learn: 0.3564697	total: 30.1s	remaining: 181ms
994:	learn: 0.3564385	total: 30.1s	remaining: 151ms
995:	learn: 0.3564092	total: 30.1s	remaining: 121ms
996:	learn: 0.3563860	total: 30.1s	remaining: 90.7ms
997:	learn: 0.3563594	total: 30.2s	remaining: 60.5ms
998:	learn: 0.3563217	total: 30.2s	remaining: 30.2ms
999:	learn: 0.3562859	total: 30.2s	remaining: 0us


In [61]:
pred[:,1]

array([0.54812147, 0.36446373, 0.16281212, ..., 0.40126401, 0.24373855,
       0.46516998])

In [63]:
#Submission dataframe
submission=pd.DataFrame({'Patient_ID':test.Patient_ID,'Health_Camp_ID':test.Health_Camp_ID,'Outcome':pred[:,1]})

submission.to_csv('votingmodel.csv',index=False)

In [ ]:
from sklearn.model_selection import KFold
kfold=KFold(n_splits=5,shuffle=True)  # shuffle true will shuffle everytime
lgbm=LGBMClassifier(n_estimators=500,max_depth=10,random_state=42,learning_rate=0.01,scale_pos_weight=3)

pred_df-pd.DataFrame()

n=5
for i in range(n):
    folds=next(kfold.split(X))  # next helps you iterate between the folds
    xtrain=X.iloc[folds[0]]  # builds xtrain data
    ytrain=y.iloc[folds[0]]   #creates y values
    lgbm.fit(xtrain,ytrain)
    pred_df[i]=lgbm.predict_proba(newtest)[:,1]

In [ ]:
median_prob=pred_df.median(axis=1)

In [ ]:
#submission
submission=pd.DataFrame({'Patient_ID':test.Patient_ID,
                        'Health_Camp_ID':test.Health_Camp_ID,
                        'Outcome':median_proba})


submission.to_csv('median_proba_model.csv',index=False)